In [37]:
import pandas as pd
import boto3
import json
from tqdm import tqdm
import ast

session = boto3.Session()

bedrock = session.client(
    service_name='bedrock-runtime',
    region_name='us-east-1',
    endpoint_url="https://bedrock-runtime.us-east-1.amazonaws.com"
)

df = pd.read_excel(r'C:\Users\joo\Desktop\aws\꾀꼬리\bedrock-tutorial\Ch12 - Automation/example.xlsx')

In [4]:
df

,수리일자,고장내용,고장부품,불량유형,조치내용
0,2023-09-07,출하장 입구 에어 50A 유니온 에어리크 재조임,50A 유니온,에어리크,재조임
1,2021-10-08,"SHIELD ROOM 화풍기 철거부위 마감,판넬벽체문타공",SHIELD ROOM 화풍기,철거부위 마감,판넬벽체문타공
2,2022-02-23,F/P 기능상실 PLC 확인 및 클램프 감지센샤 조정,NaN,NaN,NaN
3,2022-04-06,와이어 전진동작이상 점검 중 정상 작동됨,NaN,NaN,NaN
4,2021-02-20,클램프기동이상 오일보충 에어 제거,NaN,NaN,NaN
5,2021-02-26,피이더 조작판 매입 콘센트 고정,NaN,NaN,NaN
6,2021-04-13,QDC 파손 클램프 교체,NaN,NaN,NaN
7,2021-07-30,광세샤 고정블록 볼트 마모 600ton 것으로 대체,NaN,NaN,NaN
8,2022-07-12,휠 회전 작동불량 점검 세기정공 AS 요청,NaN,NaN,NaN
9,2022-07-13,클러치 ASS'Y 작동이상 분해출문(세기정공),NaN,NaN,NaN


In [5]:
sub_df = df[df['고장부품'].isnull()]
sub_df.reset_index(drop=True, inplace=True)
sub_df

,수리일자,고장내용,고장부품,불량유형,조치내용
0,2022-02-23,F/P 기능상실 PLC 확인 및 클램프 감지센샤 조정,NaN,NaN,NaN
1,2022-04-06,와이어 전진동작이상 점검 중 정상 작동됨,NaN,NaN,NaN
2,2021-02-20,클램프기동이상 오일보충 에어 제거,NaN,NaN,NaN
3,2021-02-26,피이더 조작판 매입 콘센트 고정,NaN,NaN,NaN
4,2021-04-13,QDC 파손 클램프 교체,NaN,NaN,NaN
5,2021-07-30,광세샤 고정블록 볼트 마모 600ton 것으로 대체,NaN,NaN,NaN
6,2022-07-12,휠 회전 작동불량 점검 세기정공 AS 요청,NaN,NaN,NaN
7,2022-07-13,클러치 ASS'Y 작동이상 분해출문(세기정공),NaN,NaN,NaN
8,2022-07-18,클러치 assy 불량으로 전자접촉기교체,NaN,NaN,NaN
9,2022-07-18,클러치 밋션 기어오일보출 ATF-3,NaN,NaN,NaN


In [14]:
def ask_claude(prompt):
    body = json.dumps({
        "prompt": "\n\nHuman: " + prompt + "\n\nAssistant:",
        "max_tokens_to_sample": 300,
        "temperature": 0,
    })

    modelId = 'anthropic.claude-v2'
    accept = 'application/json'
    contentType = 'application/json'

    response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)

    response_body = json.loads(response.get('body').read())

    # text
    return response_body.get('completion')

In [26]:
def make_prompt(text):
    prompt = '''주어진 고장내용 텍스트로부터 고장부품, 불량유형, 조치사항을 json 형태로 추출하시오.
    예시는 다음과 같습니다.
    
    고장내용: '출하장 입구 에어 50A 유니온 에어리크 재조임'
    출력: {"고장부품": "50A 유니온", "불량유형" : "에어리크", "조치내용" : "재조임"}

    고장내용: '출하장 입구 에어 50A 유니온 에어리크 재조임'
    출력: {"고장부품": "SHIELD ROOM 화풍기", "불량유형" : "철거부위 마감", "조치내용" : "판넬벽체문타공"}

    자, 실전입니다. 아래의 고장내용을 json 형태로 추출해보세요.
    
    고장내용: ''' + str(text) + '''
    '''
    return prompt

In [27]:
sub_df['고장내용'].loc[0]

'F/P 기능상실 PLC 확인 및 클램프 감지센샤 조정'

In [28]:
# 프롬프트 예시
print(make_prompt(sub_df['고장내용'].loc[0]))

주어진 고장내용 텍스트로부터 고장부품, 불량유형, 조치사항을 json 형태로 추출하시오.
    예시는 다음과 같습니다.
    
    고장내용: '출하장 입구 에어 50A 유니온 에어리크 재조임'
    출력: {"고장부품": "50A 유니온", "불량유형" : "에어리크", "조치내용" : "재조임"}

    고장내용: '출하장 입구 에어 50A 유니온 에어리크 재조임'
    출력: {"고장부품": "SHIELD ROOM 화풍기", "불량유형" : "철거부위 마감", "조치내용" : "판넬벽체문타공"}

    자, 실전입니다. 아래의 고장내용을 json 형태로 추출해보세요.
    
    고장내용: F/P 기능상실 PLC 확인 및 클램프 감지센샤 조정
    


In [33]:
test_prompt = make_prompt(sub_df['고장내용'].loc[0])
ask_claude(test_prompt)

' {"고장부품": "F/P", "불량유형": "기능상실", "조치내용": "PLC 확인 및 클램프 감지센샤 조정"}'

In [34]:
result = []
for sample in tqdm(sub_df['고장내용'].to_list()):
    prompt = make_prompt(sample)
    result.append(ask_claude(prompt))

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [01:03<00:00,  3.99s/it]


In [35]:
result

[' {"고장부품": "F/P", "불량유형": "기능상실", "조치내용": "PLC 확인 및 클램프 감지센샤 조정"}',
 ' {\n  "고장부품": "와이어",\n  "불량유형": "전진동작이상",\n  "조치내용": "점검 중 정상 작동됨"\n}',
 ' {"고장부품": "클램프기동", "불량유형": "이상", "조치내용": "오일보충 에어 제거"}',
 ' {"고장부품": "피이더 조작판", "불량유형": "매입 콘센트", "조치내용": "고정"}',
 ' {\n  "고장부품": "클램프",\n  "불량유형": "파손",\n  "조치내용": "교체"\n}',
 ' {\n  "고장부품": "고정블록 볼트",\n  "불량유형": "마모", \n  "조치내용": "600ton 것으로 대체"\n}',
 ' {"고장부품": "휠", "불량유형": "회전 작동불량", "조치내용": "점검 세기정공 AS 요청"}',
 ' {"고장부품": "클러치 ASS\'Y", "불량유형": "작동이상", "조치내용": "분해출문(세기정공)"}',
 ' {"고장부품": "클러치 assy", "불량유형": "불량", "조치내용": "전자접촉기교체"}',
 ' {\n  "고장부품": "클러치 밋션", \n  "불량유형": "기어오일보출",\n  "조치내용": "ATF-3"\n}',
 ' {\n  "고장부품": "휨 가지센서",\n  "불량유형": "단선",\n  "조치내용": "교체"\n}',
 ' {\n  "고장부품": "리미트캠스위치",\n  "불량유형": "타발정지위치이상",\n  "조치내용": "리미트캠스위치교체및조정,상사점리미트스위치교체"\n}',
 ' {"고장부품": "QDC 클램프", "불량유형": "기동이상", "조치내용": "에어빼기"}',
 ' {"고장부품": "피이더", "불량유형": "진입이상", "조치내용": "캠 스위치 조정"}',
 ' {"고장부품": "카운터", "불량유형": "정지시점 불량", "조치내용": "카운타 각도 조정"}',
 ' {\n  "고장

In [41]:
# 앞 공백이나 줄바꿈 제거
result = [r.strip().replace('\n', '') for r in result]

In [42]:
result

['{"고장부품": "F/P", "불량유형": "기능상실", "조치내용": "PLC 확인 및 클램프 감지센샤 조정"}',
 '{  "고장부품": "와이어",  "불량유형": "전진동작이상",  "조치내용": "점검 중 정상 작동됨"}',
 '{"고장부품": "클램프기동", "불량유형": "이상", "조치내용": "오일보충 에어 제거"}',
 '{"고장부품": "피이더 조작판", "불량유형": "매입 콘센트", "조치내용": "고정"}',
 '{  "고장부품": "클램프",  "불량유형": "파손",  "조치내용": "교체"}',
 '{  "고장부품": "고정블록 볼트",  "불량유형": "마모",   "조치내용": "600ton 것으로 대체"}',
 '{"고장부품": "휠", "불량유형": "회전 작동불량", "조치내용": "점검 세기정공 AS 요청"}',
 '{"고장부품": "클러치 ASS\'Y", "불량유형": "작동이상", "조치내용": "분해출문(세기정공)"}',
 '{"고장부품": "클러치 assy", "불량유형": "불량", "조치내용": "전자접촉기교체"}',
 '{  "고장부품": "클러치 밋션",   "불량유형": "기어오일보출",  "조치내용": "ATF-3"}',
 '{  "고장부품": "휨 가지센서",  "불량유형": "단선",  "조치내용": "교체"}',
 '{  "고장부품": "리미트캠스위치",  "불량유형": "타발정지위치이상",  "조치내용": "리미트캠스위치교체및조정,상사점리미트스위치교체"}',
 '{"고장부품": "QDC 클램프", "불량유형": "기동이상", "조치내용": "에어빼기"}',
 '{"고장부품": "피이더", "불량유형": "진입이상", "조치내용": "캠 스위치 조정"}',
 '{"고장부품": "카운터", "불량유형": "정지시점 불량", "조치내용": "카운타 각도 조정"}',
 '{  "고장부품": "NC FEEDER 코일",  "불량유형": "이송거리 이상",  "조치내용": "재 셋팅"}']

In [43]:
result = [ast.literal_eval(elem) for elem in result]
print(result)

[{'고장부품': 'F/P', '불량유형': '기능상실', '조치내용': 'PLC 확인 및 클램프 감지센샤 조정'}, {'고장부품': '와이어', '불량유형': '전진동작이상', '조치내용': '점검 중 정상 작동됨'}, {'고장부품': '클램프기동', '불량유형': '이상', '조치내용': '오일보충 에어 제거'}, {'고장부품': '피이더 조작판', '불량유형': '매입 콘센트', '조치내용': '고정'}, {'고장부품': '클램프', '불량유형': '파손', '조치내용': '교체'}, {'고장부품': '고정블록 볼트', '불량유형': '마모', '조치내용': '600ton 것으로 대체'}, {'고장부품': '휠', '불량유형': '회전 작동불량', '조치내용': '점검 세기정공 AS 요청'}, {'고장부품': "클러치 ASS'Y", '불량유형': '작동이상', '조치내용': '분해출문(세기정공)'}, {'고장부품': '클러치 assy', '불량유형': '불량', '조치내용': '전자접촉기교체'}, {'고장부품': '클러치 밋션', '불량유형': '기어오일보출', '조치내용': 'ATF-3'}, {'고장부품': '휨 가지센서', '불량유형': '단선', '조치내용': '교체'}, {'고장부품': '리미트캠스위치', '불량유형': '타발정지위치이상', '조치내용': '리미트캠스위치교체및조정,상사점리미트스위치교체'}, {'고장부품': 'QDC 클램프', '불량유형': '기동이상', '조치내용': '에어빼기'}, {'고장부품': '피이더', '불량유형': '진입이상', '조치내용': '캠 스위치 조정'}, {'고장부품': '카운터', '불량유형': '정지시점 불량', '조치내용': '카운타 각도 조정'}, {'고장부품': 'NC FEEDER 코일', '불량유형': '이송거리 이상', '조치내용': '재 셋팅'}]


In [45]:
print(type(result[0]))

<class 'dict'>


In [46]:
new_df = pd.DataFrame(result)

In [47]:
sub_df['고장부품'] = new_df['고장부품']
sub_df['불량유형'] = new_df['불량유형']
sub_df['조치내용'] = new_df['조치내용']

C:\Users\joo\AppData\Local\Temp\ipykernel_44936\2419304372.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['고장부품'] = new_df['고장부품']
C:\Users\joo\AppData\Local\Temp\ipykernel_44936\2419304372.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['불량유형'] = new_df['불량유형']
C:\Users\joo\AppData\Local\Temp\ipykernel_44936\2419304372.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

In [48]:
sub_df

,수리일자,고장내용,고장부품,불량유형,조치내용
0,2022-02-23,F/P 기능상실 PLC 확인 및 클램프 감지센샤 조정,F/P,기능상실,PLC 확인 및 클램프 감지센샤 조정
1,2022-04-06,와이어 전진동작이상 점검 중 정상 작동됨,와이어,전진동작이상,점검 중 정상 작동됨
2,2021-02-20,클램프기동이상 오일보충 에어 제거,클램프기동,이상,오일보충 에어 제거
3,2021-02-26,피이더 조작판 매입 콘센트 고정,피이더 조작판,매입 콘센트,고정
4,2021-04-13,QDC 파손 클램프 교체,클램프,파손,교체
5,2021-07-30,광세샤 고정블록 볼트 마모 600ton 것으로 대체,고정블록 볼트,마모,600ton 것으로 대체
6,2022-07-12,휠 회전 작동불량 점검 세기정공 AS 요청,휠,회전 작동불량,점검 세기정공 AS 요청
7,2022-07-13,클러치 ASS'Y 작동이상 분해출문(세기정공),클러치 ASS'Y,작동이상,분해출문(세기정공)
8,2022-07-18,클러치 assy 불량으로 전자접촉기교체,클러치 assy,불량,전자접촉기교체
9,2022-07-18,클러치 밋션 기어오일보출 ATF-3,클러치 밋션,기어오일보출,ATF-3


In [49]:
# 아래의 코드로 새 파일을 저장
# sub_df.to_excel('new_data.xlsx', index=False)